In [93]:
# !pip install arabic-stopwords
import numpy as np
import pandas as pd
import arabicstopwords.arabicstopwords as ast
import re
import string
import qalsadi.lemmatizer
from nltk.stem.isri import ISRIStemmer
from tashaphyne.stemming import ArabicLightStemmer

In [92]:
!pip install Tashaphyne

You should consider upgrading via the 'c:\users\raghod\anaconda3\python.exe -m pip install --upgrade pip' command.


# Read train data

In [94]:
train_data = pd.read_csv('./DataSet/dev.csv',sep=',',header=0)
train_data.head(10)

,text,category,stance
0,#مريم_رجوي: <LF>حظر خامنئي المجرم شراء #لقاح_ك...,info_news,1
1,#الصحة:<LF>•تم إعطاء 259.530 جرعة من لقاح #كور...,plan,1
2,#خادم_الحرمين - حفظه الله - يتلقى الجرعة الأول...,celebrity,1
3,#الصحه_العالميه: لقاحات #كورونا آمنة ولا خوف م...,info_news,1
4,"#وزيرة_الصحة ""#هالة_زايد"" تقول إنه يجرى مراجعة...",info_news,1
5,2️⃣ وانتهى الفريق من الدراسات قبل السريرية ونش...,info_news,1
6,عاجل 🔴 <LF>.<LF><LF>.<LF><LF>وزارة الصحة :<LF>...,plan,1
7,#فيديو | السفير الأميركي لدى السعودية بعد تلقي...,info_news,1
8,تصريحات وبس الحكومة مع السيسي علي حسب اللقطة! ...,info_news,0
9,الاتحاد الاوروبي تفاوض لشراء لقاحات الكورونا م...,info_news,1


In [82]:
# st = ISRIStemmer()
# w= 'حركات'
# print(train_data["text"][0])
# print(' '.join(st.stem(word) for word in train_data["text"][0].split()))

In [15]:
#remove first row that has the header
#train_data.drop(0, axis=0, inplace=True)
train_data['category'] = train_data['category'].astype('category').cat.codes
train_data.head(10)

,text,category,stance
0,مريمرجوي حظر خامنئي المجرم شراء لقاحكورونا يعد...,2,1
1,الصحة •تم إعطاء 259530 جرعة لقاح كورونا •أهلية...,5,1
2,خادمالحرمين حفظه اله يتلقى الجرعة الأولى لقاح ...,1,1
3,الصحهالعالميه لقاحات كورونا آمنة خوف كوفاكس يض...,2,1
4,وزيرةالصحة هالةزايد تقول يجرى مراجعة خطوط الإن...,2,1
5,2️⃣ وانتهى الفريق الدراسات السريرية ونشر البحث...,2,1
6,عاجل وزارة الصحة حصل الجرعة الأولى لقاح كورونا...,5,1
7,فيديو السفير الأميركي السعودية تلقيه لقاح كورن...,2,1
8,تصريحات الحكومة السيسي القطة مجاني لقاح كورونا...,2,0
9,الاتحاد الاوروبي تفاوض لشراء لقاحات الكورونا ا...,2,1


# Pre-Processing the tweets

In [16]:
#getting a stopwords_list
stop_words = ast.stopwords_list()

In [97]:
punctuations_list = '''^_-`$%&÷×؛<=>()*&^%][،/;:"؟.,'{}~¦+|!”…“'''
#english_punctuations = string.punctuation
#print(english_punctuations)
#punctuations_list = arabic_punctuations

def remove_punctuations(text):
    translator = str.maketrans(punctuations_list, ' '*len(punctuations_list))
    return text.translate(translator)
    
def remove_emoji(text):
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text) # no emoji

# a small function to remove stop words
def remove_stop_words(text):
    return ' '.join(word for word in text.split() if word not in stop_words)

# a small function to remove stop words
ArListem = ArabicLightStemmer()
def lemmatiz_word(text):
    # lemmer = qalsadi.lemmatizer.Lemmatizer()
    # return ' '.join(lemmer.lemmatize(word) for word in text.split()) 
    #---------
    #st = ISRIStemmer()
    #return ' '.join(st.stem(word) for word in text.split())
    return ' '.join(ArListem.light_stem(word) for word in text.split())
    
def processPost(tweet): 

    #Remove <LF> from tweet
    tweet = re.sub('<LF>', ' ', tweet)
    
    #Replace @username with empty string
    tweet = re.sub('@[^\s]+', ' ', tweet)
    
    #remove url
    tweet = re.sub('(https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9]+\.[^\s]{2,}|www\.[a-zA-Z0-9]+\.[^\s]{2,})',' ',tweet)
    
    #remove hashtage #
    tweet = re.sub(r'#([^\s]+)', r'\1', tweet)

    # remove punctuations
    tweet = remove_punctuations(tweet)
    
    # remove emoji  (not sure with this step)
    tweet = remove_emoji(tweet)
    
    # normalize the tweet
    # tweet= normalize_arabic(tweet)
    
    # remove repeated letters
    tweet=re.sub(r'(.)\1+', r'\1', tweet)

    #remove stop words
    tweet = remove_stop_words(tweet)

    tweet=lemmatiz_word(tweet)
    
    return tweet

In [98]:
print(train_data["text"][9])
train_data["text"] = train_data['text'].apply(lambda x: processPost(x))
train_data.head(10)
print(train_data["text"][9])

الاتحاد الاوروبي تفاوض لشراء لقاحات الكورونا الشركات المختلفة كتلة واحدة الاسعار الي حصلو جرعة مطعوم فايزر بيونتك ١٤ ٨ استرازانيكا اكسفورد ٢ ١٩ موديرنا ١٨ سانوفي ٩ ٣٠ جونسون اند جونسون ٨ ٥
اتحاد اوروب فاوض شراء قاح كورون شرك مختلف تل احد اسعار لي حصل جرع مطعوم يزر يون ١٤ ٨ استرازانيكا اكسفورد ٢ ١٩ مودير ١٨ سانوف ٩ ٣٠ جونس ند جونس ٨ ٥


# Feature Extraction 
1. TF-IDF 
2. CBoW
3. Bag of Words

In [105]:
all_words = []
for sentence in train_data["text"]:
    all_words += sentence.split(' ')

vocab = set(all_words)